# 导入模块

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,datetime
import re
import gc

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 导入订单数据

In [12]:

def select_data():
    sql1 = ''' -- 订单&风控信息  近10日数据   
    SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money ,om.has_order_check
    ,om.status, om.user_id, om.alipay_order_id
    ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
    when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
    when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单"  else "未知状态" end  as status2 
    ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
    ,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
    ,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
    ,om.user_mobile,tmu.true_name,tmu.id_card_num
    ,top.total_describes,tor.decision_result,om.cancel_reason
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
    ,cc.name as channel_name         -- 来源渠道
    ,cc.channel_type_id              -- 渠道id
    ,pa.name as activity_name        -- 活动名称
    ,om.merchant_id,om.merchant_name
    ,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
    ,om.buy_service_product,tso.status as service_status 
    ,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
    ,tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
    ,om.order_type, om.union_rent_tag, tprm.rent_received 已收租金 
    ,tolog.status 物流状态
    ,replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk
    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.union_rent_rejected') end,'"','') as union_rent_rejected   # 联合运营

    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241011') end,'"','') as strategy_241011                    # 拒量
    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241205_rejected') end,'"','') as strategy_241205_rejected  # 拒量
    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_241212_rejected') end,'"','') as strategy_241212_rejected  # 拒量
    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250120_rejected') end,'"','') as strategy_250120_rejected  # 拒量
    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.strategy_250330_rejected') end,'"','') as strategy_250330_rejected  # 拒量
    , replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.can_ship_flag') end, '"', '') as risk_can_ship_flag
    , tp.classify_id, tpmn.name 机型, tp.id 商品ID
    ,rarr.data rarr_data, rarr.time rarr_time
    ,rarrc.data_3_h, rarrc.data_6_h, rarrc.data_12_h, rarrc.data_24_h
    ,op.pay_date,ol.go_express_date
    ,tocr.change_type, tocr.支付时间
    from db_digua_business.t_order  om
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id
    -- 备注信息合并 
    left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
    on om.id = top.order_id 
    -- 服务信息
    left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
    -- 渠道名称
    left join db_digua_business.t_channel cc on om.channel = cc.scene 
    -- 活动名称
    left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
    -- 用户信息 
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    -- 商品信息
    left join db_digua_business.t_order_details tod on om.id = tod.order_id
    -- 免押信息  
    left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
    from db_digua_business.t_order_pay t 

    where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
    on topay.order_id=om.id   and  topay.rn = 1 
    -- 商家订单转移表
    left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
    -- 京东外部订单关联表
    left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
    -- 订单物流表
    left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
    -- 租后分期表
    -- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
    left join db_credit.risk risk on risk.order_id=om.id
    -- 商品表
    left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB

    left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
    -- left join db_credit.order_association oa on oa.order_id=om.id
    left join db_credit.risk_alipay_rent_risk rarr on rarr.out_biz_no = om.order_number
    left join db_credit.risk_alipay_rent_risk_cron rarrc on rarrc.out_biz_no = om.order_number
    left join db_digua_business.t_order_pay op on op.order_id = om.id and op.pay_type = 'ZFBYSQ' and op.sync_mini_order = 'Y'
    left join db_digua_business.t_order_logistics ol on om.id = ol.order_id 
    left join (select  order_id, change_type,  create_time 支付时间  from db_digua_business.t_order_change_record 
    where change_type='PASS_TIME') tocr on om.id = tocr.order_id


    where om.user_mobile is not null 
    -- and  om.create_time >= CURDATE()             -- 当天数据
    -- and  om.create_time < DATE_ADD(CURDATE() ,INTERVAL 1 day )
    and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-11-07'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-12-21'
    -- and tp.id = 6687
    -- and tp.id in (8946, 8947, 8948, 8949, 8950, 8951, 8952, 8953, 8954, 8955, 8956, 8957, 8958, 8959, 8600, 8601,8602,8603,8734,8733,8759,8758,7681,7682)
    
    '''
    
    # sql3 = '''
    # SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
    # '''

    df_order = clean.query(sql1)
    # df_risk_examine = clean.query(sql3)
    
    sql_risk = ''' -- risk等级
                        select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
                        from db_credit.risk
                        '''
    df_risk = clean.query(sql_risk)
    sql3 = '''
                SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
                '''
    df_risk_examine = clean.query(sql3)
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date, errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

    sql_ra = ''' -- 996强拒表
                select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
                '''
    df_ra = clean.query(sql_ra)
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date, errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
    
    sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
    df_name = clean.query(sql_name)
    
    sql_rd = ''' -- 顶替原来的出库前风控强拒，实际上是发货前出库强拒 2025-12-03
                select order_id, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_rd
                from db_credit.risk_delivery
        '''
    df_rd = clean.query(sql_rd)
    
    
    
    return df_order, df_risk_examine, df_risk, df_re, df_ra, df_name, df_rd

In [13]:
def drop_merchant(df):
        '''
        删除商家数据
        :param df: 传入带有商家的数据
        :return: 返回剔除了商家的数据 将在进件剔除商家数据
        '''
        # 剔除商家数据只保留自营租机业务数据
        df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)

        df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
        df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        # 趣智数码  单
        df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
        
        df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "呱子笔记本电脑"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "南京聚格网络科技"].index, inplace=True)
        
        df.drop(df[df['merchant_name'] == "星晟数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "蘑菇时间"].index, inplace=True)
        # df.drop(df[df['merchant_name'] == "云启德曜"].index, inplace=True) # 拒量
        df.drop(df[df['merchant_name'] == "艾欧尼亚数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "谦莲科技"].index, inplace=True)
        reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机", "云启德曜", "智云享租"]
        df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)

        df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        return df

In [14]:
def order_drop_duplicates(df):
        dict_status_code = {
            "订单取消": 1,
            "待支付": 2,
            "已退款": 3,
            "待审核": 4,
            "待发货": 5,
            "待收货": 6,
            "租赁中": 7,
            "已完成": 8
        }
        df["状态编码"] = df["status2"].map(dict_status_code)
        df.sort_values(by=["下单日期", "状态编码"], inplace=True)
        # order_id按rarr_time升序排序，这边存在一定的问题，理论上应该在以上排序的基础上更改排序逻辑，应以["下单日期", "状态编码", "order_id", "rarr_time"]进行排序，但既然结果
        # 符合预期，则不进行更改。
        # df.sort_values(by=["下单日期", "状态编码","order_id", "rarr_time"], inplace=True)
        
        # 删除重复单号
        df.drop_duplicates(subset=["order_id"], inplace=True)# TODO
        # 长租严选不论曙光计划，一切都需统计，故不剔除曙光计划，曙光计划可能包含身份证为空值，各种1000单计划活动，线下门店。
        # 删除身份证空值行
        # df.dropna(subset=["id_card_num"], axis=0, inplace=True)
        # 去刷单订单
        # df.drop(df[df['total_describes'].str.contains(pat='panli', regex=False) == True].index, inplace=True)
        # 曙光计划和线下小店是同分异构体
        # df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
        # df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
        # df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
        # df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
        # 删除订单状态空值行 实际上是去除status=16，待赔偿状态的订单
        df.dropna(subset=["status2"], axis=0, inplace=True) #TODO 2025-1210
        
        # 删除重复订单
        df.drop_duplicates(subset=["order_id"], inplace=True)#TODO
        # df.drop_duplicates(subset=["true_name", "user_mobile", "id_card_num", "下单日期"], keep="last", inplace=True)# 长租严选不需要对当天多次下单的数据进行去重
        return df

# 数据处理

In [15]:
def process_data(df_order, df_risk, df_re, df_ra, df_rd):
    # 复制数据
    df = df_order.copy()
    # df = df[df.type!=4]
    # print(df.order_number=='A202511121654081155')
    
    # 处理日期
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    print(df["下单日期"].unique())
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    # 订单支付时间处理
    # df['pay_date'] = df['pay_date'].dt.date
    # df['go_express_date'] = df['go_express_date'].dt.date
    # 可能需要将48h内改为下单后2个自然日内。
    df.loc[:, '2天内发货数'] = np.where((df.go_express_date.dt.date <= (df.create_time.dt.date + pd.Timedelta(days=1))) & (df.change_type == 'PASS_TIME'), 1, 0)
    # df.loc[:, '48h内发货数'] = np.where((df.status.isin([3, 4])) & (df.go_express_date - df.pay_date <= pd.Timedelta(hours=48)), 1, 0)
    
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

    df['拒绝理由_risk']= df['rejected'].fillna('')+df['strategy_241205_rejected'].fillna('')+df['strategy_241212_rejected'].fillna('')+df['strategy_241011'].fillna('')+df['strategy_250120_rejected'].fillna('')+df['strategy_250330_rejected'].fillna('')+df['union_rent_rejected'].fillna('')+df['rejected_ato_risk'].fillna('')

    df.loc[:, '拒绝理由'] = np.where(df['拒绝理由'] == '', df['拒绝理由_risk'], df['拒绝理由'])
    df['拒绝理由'] = df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')
    # 确保 total_describes 列为字符串类型
    # df["total_describes"] = df["total_describes"].astype(str).fillna("")
    # 暂时修改一下，由于 AttributeError: Can only use .str accessor with string values! 错误
    # df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    # df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    # df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    # 当order_id相同时，将所有记录的total_describes 进行拼接成一个字符串并赋值给total_describes
    # 第一步：当 order_id 相同时，将所有记录的 total_describes 拼接成一个字符串并赋值给备注
    # 第一步：拼接 total_describes
    df['total_describes'] = df['total_describes'].fillna('')
    # df.loc[:, '备注'] = df.groupby('order_id')['total_describes'].transform(
    #     lambda x: ','.join(x)
    # )

    # 第二步：提取并重新拼接 msg 内容
    df.loc[:, '备注'] = df.groupby('order_id')['total_describes'].transform(
        lambda x: ','.join(
            [msg for text in x for msg in 
            __import__('re').findall(r'msg=(.*?)"', text) if msg]
        )
    )
    
    # 更安全的处理方式
    df["取消原因"] = df["total_describes"].apply(lambda x: x.split("客户申请取消：")[1].split("$")[0].strip() if "客户申请取消：" in x and len(x.split("客户申请取消：")) > 1 else None)
    df["电审拒绝原因"] = df["total_describes"].apply(lambda x: x.split("审核不通过：")[1].split("$")[0].strip() if "审核不通过：" in x and len(x.split("审核不通过：")) > 1 else None)
    df["无法联系原因"] = df["total_describes"].apply(lambda x: x.split("用户无法联系：")[1].split("$")[0].strip() if "用户无法联系：" in x and len(x.split("用户无法联系：")) > 1 else None)
    
    
    df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
    df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
    df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
    df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
    df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")
    # df.loc[:,'can_ship_flag'] = df['rarr_data'].apply(
    # lambda json_str: 
    #     json.loads(json_str)['ship_goods_risk_models'][0]['can_ship_flag'] 
    #     if len(json.loads(json_str).get('ship_goods_risk_models', [])) > 0 
    #     else None)
    df['can_ship_flag'] = df['rarr_data'].apply(lambda x: json.loads(x).get('ship_goods_risk_models', [{}])[0].get('can_ship_flag') if pd.notna(x) and x else None)
    # 需要补充的逻辑
    df['is_bf_3h'] = df['data_3_h'].apply(lambda x: json.loads(x).get('ship_goods_risk_models', [{}])[0].get('can_ship_flag') if pd.notna(x) and x else None)
    df['is_bf_3h'] = df['is_bf_3h'].fillna(True)
    df['is_bf_6h'] = df['data_6_h'].apply(lambda x: json.loads(x).get('ship_goods_risk_models', [{}])[0].get('can_ship_flag') if pd.notna(x) and x else None)
    df['is_bf_6h'] = df['is_bf_6h'].fillna(True)
    df['is_bf_12h'] = df['data_12_h'].apply(lambda x: json.loads(x).get('ship_goods_risk_models', [{}])[0].get('can_ship_flag') if pd.notna(x) and x else None)
    df['is_bf_12h'] = df['is_bf_12h'].fillna(True)
    # 2025-12-19 新增data_24_h 字段,在此之前判断的优质订单均存在问题
    df['is_bf_24h'] = df['data_24_h'].apply(lambda x: json.loads(x).get('ship_goods_risk_models', [{}])[0].get('can_ship_flag') if pd.notna(x) and x else None)
    df['is_bf_24h'] = df['is_bf_24h'].fillna(True)
    
    # 处理备注信息
    # df['取消原因2'] = df['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
    def getcolor(s):
        color_list=json.loads(s)
        for j in range(0, len(color_list)):
            if color_list[j]["key"]=="颜色":
                return color_list[j]["value"]
    df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)


    def getneicun(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="内存":
                return color_list[j]["value"]

    df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
    def getfangan(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="租赁方案":
                return color_list[j]["value"]
    # df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)

    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")


    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
    df.loc[:,'机型内存'] = df['机型'] +'_'+ df['内存']
    df.loc[:, '机型内存颜色'] = df['机型'] +'_'+ df['内存'] +'_'+ df['颜色']
    
    
    df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
    
    df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
    
    
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left').merge(df_rd, on='order_id', how='left')
    # 定义状态
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
    # df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
    # df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
    
    # df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
    # df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
    df['是否机审强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1')|(df.status_r=='1'), 1, 0)
    df['是否出库前风控强拒'] = np.where(df.status_rd=='1', 1, 0)
    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
    # 订单去重
    df = order_drop_duplicates(df)
    
    # 活动商品ID
    old_product_id_list = [8946, 8947, 8948, 8949, 8950, 8951, 8952, 8953, 8954, 8955, 8956, 8957, 8958, 8959, 8600, 8601,8602,8603,8734,8733,8759,8758,7681,7682]
    product_id_list = [8946, 8947, 8948, 8949, 8950, 8951, 8952, 8953, 8954, 8955, 8956, 8957, 8958, 8959,8734,8733,8759,8758,7682,7681]
    new_product_id_list = [7934, 7933, 7931, 7937, 7936, 7935, 7938, 7939, 7940]# 2025-11-25新增
    df.loc[:, '严选活动商品'] = np.where(df.商品ID.isin(old_product_id_list), '是', '否')
    df.loc[:, '严选活动商品'] = np.where((~df.商品ID.isin(product_id_list))&(df.下单日期>='2025-11-15'), '否', df.严选活动商品)
    df.loc[:, '严选活动商品'] = np.where((df.商品ID.isin(new_product_id_list))&(df.下单日期>'2025-11-25'), '是', df.严选活动商品)
    
    
    # 保留、剔除商家数据
    df_contain = df.copy()
    # df = drop_merchant(df) # 长租严选需要加上商家，支付宝不论商家，仅看严选活动与必发标TODO
    
    # 去重订单明细导出
    # today = datetime.today() 
    # t_date = today.date().strftime('%Y-%m-%d')
    
    # df_j = df[df["进件"]=="进件"]
    
    df = clean.status_node(df)
    
    # 剔除拒量订单
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量==0]
    # day_name_cn_mapping = {
    #     'Monday': '星期一',
    #     'Tuesday': '星期二',
    #     'Wednesday': '星期三',
    #     'Thursday': '星期四',
    #     'Friday': '星期五',
    #     'Saturday': '星期六',
    #     'Sunday': '星期日'
    # }
    
    return df, df2

## 查找订单出库前必发标签状态(多半失效需要修改)

In [16]:
# def select_bf_data():
#     sql_yx = '''
#     select
#     out_biz_no, time, data bf_data
#     from db_credit.risk_alipay_rent_risk 
#     where date_format(time,'%Y-%m-%d') >= '2025-11-07'

#     '''
#     df_yx = clean.query(sql_yx)
#     # 按out_biz_no分组挑选time最大的订单记录
#     df_yx_ckq = df_yx.sort_values('time').drop_duplicates('out_biz_no', keep='last')
#     df_yx_ckq['is_bf'] = df_yx_ckq['bf_data'].apply(lambda x: json.loads(x).get('ship_goods_risk_models', [{}])[0].get('can_ship_flag') if pd.notna(x) and x else None)
    
#     return df_yx_ckq
    

# 定时任务

In [17]:

# from apscheduler.schedulers.blocking import BlockingScheduler
# from apscheduler.schedulers.background import BackgroundScheduler
# import gcs


# # H = datetime.now().strftime('%H')
# # M = datetime.now().strftime('%M')
# def my_job1():
    
#     # print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = r'\\digua\迪瓜租机\002数据监测\3.转化数据\长租严选商品/'
#     Today1 = str(datetime.now().strftime('%Y%m%d%H%M'))
#     print(f'执行定时任务：现在是{Today1}')
    
#     df_order, df_risk_examine, df_risk, df_re, df_ra, df_name, df_rd = select_data()
    
#     df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
#     # 创建分配人映射关系 
#     name_dict = {
#             '小张': '李巧玲',
#             '小周': '李巧凤',
#             '小南': '刘三妹',
#             '小何': '何静',
#             '小谢': '谢金凤',
#             '小咪': '杨健',
#             '小慧': '林思慧',
#             '小滢': '胡彩滢',
#             '小星': '廖丽敏',
#             '小兰': '黄兰娟',
#             '小晚': '周莹',
#             '小芳': '罗芳'
#             }
#     name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏', '黄兰娟', '周莹', '邹巧巧', '冯二洋','罗芳', '魏朵','周念慈','洪佳祥']

#     # 匹配映射字典
#     df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
#     df_name = df_name[df_name.信审员.isin(name_list)]
#         # df_risk_examine = query(sql3)
#         # 数据处理
#     df_all, df2_all = process_data(df_order, df_risk, df_re, df_ra, df_rd)   
#     # 活动商品ID
#     # product_id_list = [8946, 8947, 8948, 8949, 8950, 8951, 8952, 8953, 8954, 8955, 8956, 8957, 8958, 8959,8734,8733,8759,8758,7682,7681]
    
#     df = df_all[df_all.严选活动商品=='是']
#     df2 = df2_all[df2_all.严选活动商品=='是']
    
#     df_group = all_models.data_group(df, df2, df_risk_examine, '下单日期')
#     df_group = df_group[["去重订单数","进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
#     
#     # 去重进件数，不对订单进行物理删除，仅做标记，标记每个首次出现的id_card_num，下单日期记录去重进件数为1，其余相同的mobile为0，2025-12-01
#     # df.loc[:, '去重进件数'] = np.where(df.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     # df_qc = df.groupby('下单日期').agg({'去重进件数':'sum'})
#     # df_group = pd.concat([df_qc['去重进件数'], df_group], axis=1)
    
#     # 必发长租严选活动商品ID
#     df_yx = df[df.can_ship_flag==True]
#     df2_yx = df2[df2.can_ship_flag==True]
#     df_group_yx = all_models.data_group(df_yx, df2_yx, df_risk_examine, '下单日期')    
#     df_group_yx = df_group_yx[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
#     
#     df_yx.loc[:, '去重进件数'] = np.where(df_yx.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     df_yx_qc = df_yx.groupby('下单日期').agg({'去重进件数':'sum'})
#     df_group_yx = pd.concat([df_yx_qc['去重进件数'], df_group_yx], axis=1)
    
    
#     # 长租严选非活动商品ID
#     df_all_yx = df_all[(df_all.严选活动商品=='否')&(df_all.can_ship_flag==True)]
#     df2_all_yx = df2_all[(df2_all.严选活动商品=='否')&(df2_all.can_ship_flag==True)]
#     df_all_group_yx = all_models.data_group(df_all_yx, df2_all_yx, df_risk_examine, '下单日期')    
#     df_all_group_yx = df_all_group_yx[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
#    
#     df_all_yx.loc[:, '去重进件数'] = np.where(df_all_yx.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     df_all_yx_qc = df_all_yx.groupby('下单日期').agg({'去重进件数':'sum'})
#     df_all_group_yx = pd.concat([df_all_yx_qc['去重进件数'], df_all_group_yx], axis=1)
    
#     # df_all.loc[:, '是否为严选活动商品'] = np.where(df_all.严选活动商品=='是', '是', '否')
#     # df_yx_ckq = select_bf_data()
#     # df_yx_ckq= df_yx.merge(df_yx_ckq[['out_biz_no', 'is_bf']], on='out_biz_no', how='left')
#     
#     df_bf = df_all[df_all.can_ship_flag==True]
#     df2_bf = df2_all[df2_all.can_ship_flag==True]
#     df_all_group = all_models.data_group(df_bf, df2_bf, df_risk_examine, '下单日期')
#     df_all_group = df_all_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                                 "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
#     df_bf.loc[:, '去重进件数'] = np.where(df_bf.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     df_bf_qc = df_bf.groupby('下单日期').agg({'去重进件数':'sum'})
#     df_all_group = pd.concat([df_bf_qc['去重进件数'], df_all_group], axis=1)
    
    
#     
#     df_yx_ckq1 = select_bf_data()
#     df_yx_ckq = df_yx.merge(df_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
#     df_yx_ckq = df_yx_ckq[(df_yx_ckq.is_bf==True)&(df_yx_ckq.下单日期>='2025-11-14')]
#     df2_yx_ckq = df2_yx.merge(df_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
#     df2_yx_ckq = df2_yx_ckq[(df2_yx_ckq.is_bf==True)&(df2_yx_ckq.下单日期>='2025-11-14')]
#     df_group_yx_ckq = all_models.data_group(df_yx_ckq, df2_yx_ckq, df_risk_examine, '下单日期')
#     df_group_yx_ckq = df_group_yx_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
#     
#     df_yx_ckq.loc[:, '去重进件数'] = np.where(df_yx_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     df_yx_ckq_qc = df_yx_ckq.groupby('下单日期').agg({'去重进件数':'sum'})
#     df_group_yx_ckq = pd.concat([df_yx_ckq_qc['去重进件数'], df_group_yx_ckq], axis=1)
    
#     # 严选必发
#     df_bf = df_all[df_all.can_ship_flag==True]
#     df_yx_qx = df_bf[(df_bf.客户取消==1) | (df_bf['status2']=='已退款')][['order_number', 'status2','归属渠道', '严选活动商品','cancel_reason', '取消原因','备注']]
#     df_bf = df_bf.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
#     df_dsh = df_bf[(df_bf.status2=='待审核')&(df_bf.can_ship_flag==True)][['order_number', '信审员', '严选活动商品']]
    
#     
#     df_all_yx_ckq1 = select_bf_data()
#     df_all_yx_ckq = df_all_yx.merge(df_all_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
#     df_all_yx_ckq = df_all_yx_ckq[(df_all_yx_ckq.is_bf==True)&(df_all_yx_ckq.下单日期>='2025-11-14')]
#     df2_all_yx_ckq = df2_all_yx.merge(df_all_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
#     df2_all_yx_ckq = df2_all_yx_ckq[(df2_all_yx_ckq.is_bf==True)&(df2_all_yx_ckq.下单日期>='2025-11-14')]
#     df_all_group_yx_ckq = all_models.data_group(df_all_yx_ckq, df2_all_yx_ckq, df_risk_examine, '下单日期')
#     df_all_group_yx_ckq = df_all_group_yx_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
#     df_all_yx_ckq.loc[:, '去重进件数'] = np.where(df_all_yx_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     df_all_yx_ckq_qc = df_all_yx_ckq.groupby('下单日期').agg({'去重进件数':'sum'})
#     df_all_group_yx_ckq = pd.concat([df_all_yx_ckq_qc['去重进件数'], df_all_group_yx_ckq], axis=1)
    
#     
#     df_all_ckq1 = select_bf_data()
#     df_all_ckq = df_all[df_all.can_ship_flag==True].merge(df_all_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
#     df_all_ckq = df_all_ckq[(df_all_ckq.is_bf==True)&(df_all_ckq.下单日期>='2025-11-14')]
#     df2_all_ckq = df2_all[df2_all.can_ship_flag==True].merge(df_all_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
#     df2_all_ckq = df2_all_ckq[(df2_all_ckq.is_bf==True)&(df2_all_ckq.下单日期>='2025-11-14')]
#     df_group_ckq = all_models.data_group(df_all_ckq, df2_all_ckq, df_risk_examine, '下单日期')
#     df_group_ckq = df_group_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
#                     "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
#     df_all_ckq.loc[:, '去重进件数'] = np.where(df_all_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
#     df_all_ckq_qc = df_all_ckq.groupby('下单日期').agg({'去重进件数':'sum', '发货数': 'sum'})
#     df_group_ckq = pd.concat([df_all_ckq_qc[['去重进件数', '发货数']], df_group_ckq], axis=1)
#     # 2025-12-08
#     # df_all_ckq_fh = df_all_ckq.ge
    
    
    
#     with pd.ExcelWriter(path1 + f'长租严选商品ID转化测试_{Today1}.xlsx', engine='openpyxl') as writer:
#         df_group.to_excel(writer, sheet_name='总体商品ID转化', index=True)
#         df_group_yx.to_excel(writer, sheet_name='严选活动必发', index=True)
#         df_all_group_yx.to_excel(writer, sheet_name='非严选活动必发', index=True)
#         df_all_group.to_excel(writer, sheet_name='下单时必发汇总', index=True)
#         df_group_yx_ckq.to_excel(writer, sheet_name='严选活动必发实时', index=True)
#         df_all_group_yx_ckq.to_excel(writer, sheet_name='非严选活动必发实时', index=True)
#         df_group_ckq.to_excel(writer, sheet_name='发货前必发汇总', index=True)
#         df_dsh.to_excel(writer, sheet_name='待审核', index=False)
#         df_yx_qx.to_excel(writer, sheet_name='取消', index=False)
        
#     del df_group_yx, df_group, df2_yx, df2, df_yx, df, df_yx_qx, df_group_yx_ckq, df_group_ckq, df_dsh,  df_all_group_yx_ckq, df_all_group_yx, df_all_group
#     gc.collect()
#     print("数据已保存,内存已回收")
#     # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#     #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
#     #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

# scheduler1 = BackgroundScheduler()
# scheduler2 = BackgroundScheduler()
# # 添加一个cron任务，每天的17点55分执行
# hour = 17
# minute = 55
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# # scheduler2.add_job(my_job1, 'cron', hour=14, minute=10)
# my_job1()

# # my_job1(Time='23:59:59')
# # 启动scheduler1、2
# scheduler1.start()
# scheduler2.start()



# 出库也含商家

In [18]:
def group_data(df0, classify):
    df_group = df0.groupby(classify).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum','出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件数', '是否出库': '出库'},inplace=True )

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["进件数"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group['进件数']
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["机审通过件"] = df_group["进件数"]-df_group["机审强拒"]

    df_group["风控通过件"] = df_group["进件数"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group['进件数']
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group['进件数']
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group['进件数']
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group['进件数']
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group['进件数']
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["出库"]/df_group['进件数']
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

# 定时任务（更改后

In [19]:

from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler
import gc

# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1():
    
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\002数据监测\3.转化数据\长租严选商品/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H%M'))
    print(f'执行定时任务：现在是{Today1}')
    
    df_order, df_risk_examine, df_risk, df_re, df_ra, df_name, df_rd = select_data()
    
    df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
    # 创建分配人映射关系 
    name_dict = {
            '小张': '李巧玲',
            '小周': '李巧凤',
            '小南': '刘三妹',
            '小何': '何静',
            '小谢': '谢金凤',
            '小咪': '杨健',
            '小慧': '林思慧',
            '小滢': '胡彩滢',
            '小星': '廖丽敏',
            '小兰': '黄兰娟',
            '小晚': '周莹',
            '小芳': '罗芳'
            }
    name_list = ['罗文龙', '何静', '刘三妹', '杨健', '林思慧', '胡彩滢', '周汉鸿', '廖丽敏', '黄兰娟', '周莹', '邹巧巧', '冯二洋','罗芳', '魏朵','周念慈','洪佳祥']

    # 匹配映射字典
    df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
    df_name = df_name[df_name.信审员.isin(name_list)]
        # df_risk_examine = query(sql3)
        # 数据处理
    df_all, df2_all = process_data(df_order, df_risk, df_re, df_ra, df_rd)  
    # 活动商品ID
    # product_id_list = [8946, 8947, 8948, 8949, 8950, 8951, 8952, 8953, 8954, 8955, 8956, 8957, 8958, 8959,8734,8733,8759,8758,7682,7681]
    
    df = df_all[df_all.risk_can_ship_flag=='true']
    df2 = df2_all[df2_all.risk_can_ship_flag=='true']
    
    df_group = group_data(df, '下单日期')
    df_group = df_group[["去重订单数","进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    # TODO 可能需要更改逻辑，原逻辑是由商品ID判断是否有阵地长租标签，现逻辑可通过risk_can_ship_flag判断
    # 必发长租严选活动商品ID
    df_yx = df[df.can_ship_flag==True]
    df2_yx = df2[df2.can_ship_flag==True]
    df_group_yx = group_data(df_yx, '下单日期')    
    df_group_yx = df_group_yx[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)

    df_yx.loc[:, '去重进件数'] = np.where(df_yx.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
    df_yx_qc = df_yx.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
    df_group_yx = pd.concat([df_yx_qc[['去重进件数', '2天内发货数']], df_group_yx], axis=1)
    
    # TODO 可能需要更改逻辑，原逻辑是由商品ID判断是否有阵地长租标签，现逻辑可通过risk_can_ship_flag判断 2025-12-23
    # 必发长租严选非活动商品ID
    # df_all_yx = df_all[(df_all.严选活动商品=='否')&(df_all.can_ship_flag==True)]
    # df2_all_yx = df2_all[(df2_all.严选活动商品=='否')&(df2_all.can_ship_flag==True)]
    # 2025-12-23 更改逻辑
    df_all_yx = df_all[(df_all.risk_can_ship_flag=='false')&(df_all.can_ship_flag==True)]
    df2_all_yx = df2_all[(df2_all.risk_can_ship_flag=='false')&(df2_all.can_ship_flag==True)]
    df_all_group_yx = group_data(df_all_yx, '下单日期')    
    df_all_group_yx = df_all_group_yx[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    df_all_yx.loc[:, '去重进件数'] = np.where(df_all_yx.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
    df_all_yx_qc = df_all_yx.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
    df_all_group_yx = pd.concat([df_all_yx_qc[['去重进件数', '2天内发货数']], df_all_group_yx], axis=1)
    
    
    # 严选下单必发不分活动非活动
    df_bf = df_all[df_all.can_ship_flag==True]
    df2_bf = df2_all[df2_all.can_ship_flag==True]
    df_all_group = group_data(df_bf, '下单日期')
    df_all_group = df_all_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                                "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    df_bf.loc[:, '去重进件数'] = np.where(df_bf.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
    df_bf_qc = df_bf.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
    df_all_group = pd.concat([df_bf_qc[['去重进件数', '2天内发货数']], df_all_group], axis=1)
    
    # (df_all_merge.can_ship_flag==True)&(df_all_merge.is_bf_3h==True)&(df_all_merge.is_bf_6h==True)&(df_all_merge.is_bf_12h==True)&(df_all_merge.is_bf_24h==True)&(df_all_merge.是否出库前风控强拒==0), 1, 0)
    # TODO 严选下单及24h内均必发且未命中发货前强拒（租安盾拦截）
    df_yx_ckq = df_yx[(df_yx.is_bf_3h==True)&(df_yx.is_bf_6h==True)&(df_yx.is_bf_12h==True)&(df_yx.is_bf_24h==True)&(df_yx.是否出库前风控强拒==0)]
    df_group_yx_ckq = group_data(df_yx_ckq, '下单日期')
    df_group_yx_ckq = df_group_yx_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    
    # # 严选下单出库均必发, 已失效
    # df_yx_ckq1 = select_bf_data()
    # df_yx_ckq = df_yx.merge(df_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
    # df_yx_ckq = df_yx_ckq[(df_yx_ckq.is_bf==True)&(df_yx_ckq.下单日期>='2025-11-14')]
    # df2_yx_ckq = df2_yx.merge(df_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
    # df2_yx_ckq = df2_yx_ckq[(df2_yx_ckq.is_bf==True)&(df2_yx_ckq.下单日期>='2025-11-14')]
    # df_group_yx_ckq = group_data(df_yx_ckq, '下单日期')
    # df_group_yx_ckq = df_group_yx_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
    #                 "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    # TODO 统计去重进件数并将其插df_group_yx_ckq的第一列
    df_yx_ckq.loc[:, '去重进件数'] = np.where(df_yx_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
    df_yx_ckq_qc = df_yx_ckq.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
    df_group_yx_ckq = pd.concat([df_yx_ckq_qc[['去重进件数', '2天内发货数']], df_group_yx_ckq], axis=1)
    
    # 严选必发
    df_bf = df_all[df_all.can_ship_flag==True]
    df_yx_qx = df_bf[(df_bf.客户取消==1) | (df_bf['status2']=='已退款')][['order_number', 'status2','归属渠道', '严选活动商品','cancel_reason', '取消原因','备注']]
    df_bf = df_bf.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
    df_dsh = df_bf[(df_bf.status2=='待审核')&(df_bf.can_ship_flag==True)][['order_number', '信审员', '严选活动商品']]
    
    
    # TODO 非严选下单及24h内均必发且未命中发货前强拒（租安盾拦截）
    df_all_yx_ckq = df_all_yx[(df_all_yx.is_bf_3h==True)&(df_all_yx.is_bf_6h==True)&(df_all_yx.is_bf_12h==True)&(df_all_yx.is_bf_24h==True)&(df_all_yx.是否出库前风控强拒==0)]
    df_all_group_yx_ckq = group_data(df_all_yx_ckq, '下单日期')
    df_all_group_yx_ckq = df_all_group_yx_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    # #  非严选下单出库均必发
    # df_all_yx_ckq1 = select_bf_data()
    # df_all_yx_ckq = df_all_yx.merge(df_all_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
    # df_all_yx_ckq = df_all_yx_ckq[(df_all_yx_ckq.is_bf==True)&(df_all_yx_ckq.下单日期>='2025-11-14')]
    # df2_all_yx_ckq = df2_all_yx.merge(df_all_yx_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
    # df2_all_yx_ckq = df2_all_yx_ckq[(df2_all_yx_ckq.is_bf==True)&(df2_all_yx_ckq.下单日期>='2025-11-14')]
    # df_all_group_yx_ckq = group_data(df_all_yx_ckq, '下单日期')
    # df_all_group_yx_ckq = df_all_group_yx_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
    #                 "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    
    df_all_yx_ckq.loc[:, '去重进件数'] = np.where(df_all_yx_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
    df_all_yx_ckq_qc = df_all_yx_ckq.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
    df_all_group_yx_ckq = pd.concat([df_all_yx_ckq_qc[['去重进件数', '2天内发货数']], df_all_group_yx_ckq], axis=1)
    
    
    # TODO 下单及24h内均必发且未命中发货前强拒（租安盾拦截），不分是否活动
    df_all_ckq = df_all[(df_all.can_ship_flag==True)&(df_all.is_bf_3h==True)&(df_all.is_bf_6h==True)&(df_all.is_bf_12h==True)&(df_all.is_bf_24h==True)&(df_all.是否出库前风控强拒==0)]
    df_group_ckq = group_data(df_all_ckq, '下单日期')
    df_group_ckq = df_group_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    
    
    # # 发货必发汇总，不分是否活动
    # df_all_ckq1 = select_bf_data()
    # df_all_ckq = df_all[df_all.can_ship_flag==True].merge(df_all_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
    # df_all_ckq = df_all_ckq[(df_all_ckq.is_bf==True)&(df_all_ckq.下单日期>='2025-11-14')]
    # df2_all_ckq = df2_all[df2_all.can_ship_flag==True].merge(df_all_ckq1[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
    # df2_all_ckq = df2_all_ckq[(df2_all_ckq.is_bf==True)&(df2_all_ckq.下单日期>='2025-11-14')]
    # df_group_ckq = group_data(df_all_ckq, '下单日期')
    # df_group_ckq = df_group_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
    #                 "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
    
    df_all_ckq.loc[:, '去重进件数'] = np.where(df_all_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
    df_all_ckq_qc = df_all_ckq.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
    df_group_ckq = pd.concat([df_all_ckq_qc[['去重进件数', '2天内发货数']], df_group_ckq], axis=1)
    # 2025-12-08
    # df_all_ckq_fh = df_all_ckq.ge
    
    
    
    with pd.ExcelWriter(path1 + f'长租严选商品ID转化TEST_{Today1}.xlsx', engine='openpyxl') as writer:
        df_group.to_excel(writer, sheet_name='总体商品ID转化', index=True)
        df_group_yx.to_excel(writer, sheet_name='严选活动必发', index=True)
        df_all_group_yx.to_excel(writer, sheet_name='非严选活动必发', index=True)
        df_all_group.to_excel(writer, sheet_name='下单时必发汇总', index=True)
        df_group_yx_ckq.to_excel(writer, sheet_name='严选活动必发实时', index=True)
        df_all_group_yx_ckq.to_excel(writer, sheet_name='非严选活动必发实时', index=True)
        df_group_ckq.to_excel(writer, sheet_name='发货前必发汇总', index=True)
        df_dsh.to_excel(writer, sheet_name='待审核', index=False)
        df_yx_qx.to_excel(writer, sheet_name='取消', index=False)
        
    del df_group_yx, df_group, df2_yx, df2, df_yx, df, df_yx_qx, df_group_yx_ckq, df_group_ckq, df_dsh,  df_all_group_yx_ckq, df_all_group_yx, df_all_group
    gc.collect()

    print("数据已保存,内存已回收")
    # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
    #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

scheduler1 = BackgroundScheduler()
scheduler2 = BackgroundScheduler()
# 添加一个cron任务，每天的17点55分执行
hour = 17
minute = 40
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
scheduler2.add_job(my_job1, 'cron', hour=14, minute=10)
my_job1()

# my_job1(Time='23:59:59')
# 启动scheduler1、2
scheduler1.start()
scheduler2.start()



执行定时任务：现在是202512231548
<DatetimeArray>
['2025-12-08 00:00:00', '2025-12-09 00:00:00', '2025-12-10 00:00:00',
 '2025-12-11 00:00:00', '2025-12-12 00:00:00', '2025-12-13 00:00:00',
 '2025-12-14 00:00:00', '2025-12-15 00:00:00', '2025-12-16 00:00:00',
 '2025-12-17 00:00:00', '2025-12-18 00:00:00', '2025-12-19 00:00:00',
 '2025-12-20 00:00:00', '2025-12-21 00:00:00', '2025-12-22 00:00:00',
 '2025-12-23 00:00:00']
Length: 16, dtype: datetime64[ns]
数据已保存,内存已回收


# 测试

In [23]:


df_order, df_risk_examine, df_risk, df_re, df_ra, df_name, df_rd = select_data()
df_all, df2_all = process_data(df_order, df_risk, df_re, df_ra, df_rd)

<DatetimeArray>
['2025-12-08 00:00:00', '2025-12-09 00:00:00', '2025-12-10 00:00:00',
 '2025-12-11 00:00:00', '2025-12-12 00:00:00', '2025-12-13 00:00:00',
 '2025-12-14 00:00:00', '2025-12-15 00:00:00', '2025-12-16 00:00:00',
 '2025-12-17 00:00:00', '2025-12-18 00:00:00', '2025-12-19 00:00:00',
 '2025-12-20 00:00:00', '2025-12-21 00:00:00', '2025-12-22 00:00:00',
 '2025-12-23 00:00:00']
Length: 16, dtype: datetime64[ns]


# 获取定时任务数据

In [10]:
df_all.商品类型.value_counts()

NameError: name 'df_all' is not defined

In [26]:
# 下单时必发、非必发，优质订单，严选活动
# df_all = df_all[df_all.商品类型=='二手']
# df2_all = df2_all[df2_all.商品类型=='二手']
# 必发长租严选活动商品ID,不分商品ID，仅看risk.data.can_ship_flag(risk_can_ship_flag==True)

df_yx = df_all[df_all.risk_can_ship_flag=='true']
df2_yx = df2_all[df2_all.risk_can_ship_flag=='true']
df_group_yx = group_data(df_yx, '下单日期')    
df_group_yx = df_group_yx[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
# TODO 统计去重进件数并将其插df_group_yx的第一列
df_yx.loc[:, '去重进件数'] = np.where(df_yx.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
df_yx_qc = df_yx.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
df_group_yx = pd.concat([df_yx_qc[['去重进件数', '2天内发货数']], df_group_yx], axis=1)

# 严选下单必发不分活动非活动
df_bf = df_all[df_all.can_ship_flag==True]
df2_bf = df2_all[df2_all.can_ship_flag==True]
df_all_group = group_data(df_bf, '下单日期')
df_all_group = df_all_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
df_bf.loc[:, '去重进件数'] = np.where(df_bf.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
df_bf_qc = df_bf.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
df_all_group = pd.concat([df_bf_qc[['去重进件数', '2天内发货数']], df_all_group], axis=1)

# 严选下单非必发不分活动非活动
df_fbf = df_all[df_all.can_ship_flag==False]
df2_fbf = df2_all[df2_all.can_ship_flag==False]
df_all_group_fbf = group_data(df_fbf, '下单日期')
df_all_group_fbf = df_all_group_fbf[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)
df_fbf.loc[:, '去重进件数'] = np.where(df_fbf.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
df_fbf_qc = df_fbf.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
df_all_group_fbf = pd.concat([df_fbf_qc[['去重进件数', '2天内发货数']], df_all_group_fbf], axis=1)





# TODO 下单及24h内均必发且未命中发货前强拒（租安盾拦截），不分是否活动
df_all_ckq = df_all[(df_all.can_ship_flag==True)&(df_all.is_bf_3h==True)&(df_all.is_bf_6h==True)&(df_all.is_bf_12h==True)&(df_all.is_bf_24h==True)&(df_all.是否出库前风控强拒==0)]
df_all_ckq = df_all_ckq[df_all_ckq.下单日期>='2025-12-18']
df_group_ckq = group_data(df_all_ckq, '下单日期')
df_group_ckq = df_group_ckq[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比']].fillna(0)


df_all_ckq.loc[:, '去重进件数'] = np.where(df_all_ckq.duplicated(subset=['下单日期', 'id_card_num'], keep='first'), 0, 1)
df_all_ckq_qc = df_all_ckq.groupby('下单日期').agg({'去重进件数':'sum', '2天内发货数': 'sum'})
df_group_ckq = pd.concat([df_all_ckq_qc[['去重进件数', '2天内发货数']], df_group_ckq], axis=1)

In [27]:
with pd.ExcelWriter(f'F:\需求\总需求/长租严选商品ID转化_20251223.xlsx', engine='openpyxl') as writer:
    df_group_yx.to_excel(writer, sheet_name='阵地严选活动必发', index=True)
    df_all_group.to_excel(writer, sheet_name='下单时必发汇总', index=True)
    df_all_group_fbf.to_excel(writer, sheet_name='下单时非必发汇总', index=True)
    df_group_ckq.to_excel(writer, sheet_name='优质订单必发汇总', index=True)

## 核对发货率数据

In [36]:
df_all = df_all[df_all.归属渠道=='芝麻租物']
df_yx_ckq = select_bf_data()

# change_type=='PASS_TIME'，初审通过，狭义上的支付成功
df_all.loc[:, '是否48h'] = np.where((df_all.go_express_date - df_all.支付时间 <= pd.Timedelta(hours=48))&(df_all.change_type=='PASS_TIME'), 1, 0)
df_all.loc[:, '七天内'] = np.where((df_all.go_express_date - df_all.支付时间 <= pd.Timedelta(days=7))&(df_all.change_type=='PASS_TIME'), 1, 0)
df_all.loc[:, '支付数'] = np.where(df_all.change_type=='PASS_TIME', 1, 0)
df_all_merge = df_all.merge(df_yx_ckq[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
df_all_merge.loc[:, '优质订单'] = np.where((df_all_merge.is_bf==True)&(df_all_merge.can_ship_flag==True), 1, 0)
# 支付笔数1	支付2天内发货率	优质订单支付笔数	优质订单支付2天内发货率	支付7天内发货笔数	支付7天内发货率	优质订单支付7天内发货笔数	优质订单支付7天内发货率
df_all_merge.loc[:, '优质订单支付笔数'] = np.where((df_all_merge.优质订单==1)&(df_all_merge.是否进件==1), 1, 0)
df_all_merge.loc[:, '支付2天内发货笔数'] = np.where((df_all_merge.是否48h==1)&(df_all_merge.是否出库==1), 1, 0)
df_all_merge.loc[:, '优质订单支付2天内发货笔数'] = np.where((df_all_merge.支付2天内发货笔数==1)&(df_all_merge.优质订单==1), 1, 0)
df_all_merge.loc[:, '支付7天内发货笔数'] = np.where((df_all_merge.七天内==1)&(df_all_merge.是否出库==1), 1, 0)
df_all_merge.loc[:, '优质订单支付7天内发货笔数'] = np.where((df_all_merge.支付7天内发货笔数==1)&(df_all_merge.优质订单==1), 1, 0)
df_all_merge_g = df_all_merge.groupby('下单日期').agg({'是否进件':'sum', '优质订单支付笔数':'sum', '优质订单支付2天内发货笔数':'sum' , '支付2天内发货笔数':'sum', '支付7天内发货笔数':'sum', '优质订单支付7天内发货笔数':'sum'})
df_all_merge_g.loc[:, '支付2天内发货率'] = (df_all_merge_g['支付2天内发货笔数']/df_all_merge_g['是否进件']).apply(lambda x: format(x,'.2%'))
df_all_merge_g.loc[:, '优质订单支付2天内发货率'] = (df_all_merge_g['优质订单支付2天内发货笔数']/df_all_merge_g['优质订单支付笔数']).apply(lambda x: format(x,'.2%'))
df_all_merge_g.loc[:, '支付7天内发货率'] = (df_all_merge_g['支付7天内发货笔数']/df_all_merge_g['是否进件']).apply(lambda x: format(x,'.2%'))
df_all_merge_g.loc[:, '优质订单支付7天内发货率'] = (df_all_merge_g['优质订单支付7天内发货笔数']/df_all_merge_g['优质订单支付笔数']).apply(lambda x: format(x,'.2%'))
df_all_merge_g.rename(columns={'是否进件':'支付笔数'}, inplace=True)
df_all_merge_g[['支付笔数','支付2天内发货笔数', '支付2天内发货率','优质订单支付笔数', '优质订单支付2天内发货笔数','优质订单支付2天内发货率', '支付7天内发货笔数', '支付7天内发货率', '优质订单支付7天内发货笔数', '优质订单支付7天内发货率']]

NameError: name 'select_bf_data' is not defined

# 核验优质订单明细数据

## 核验优质订单双方总数、找出各自的差异订单

In [115]:
df_all.下单日期.unique()

<DatetimeArray>
['2025-12-01 00:00:00', '2025-12-02 00:00:00', '2025-12-03 00:00:00',
 '2025-12-04 00:00:00', '2025-12-05 00:00:00', '2025-12-06 00:00:00',
 '2025-12-07 00:00:00', '2025-12-08 00:00:00', '2025-12-09 00:00:00',
 '2025-12-10 00:00:00']
Length: 10, dtype: datetime64[ns]

In [94]:
df_ali_order = pd.read_excel('F:\需求\总需求\迪瓜租机阵地发货数据核对.xlsx')
df_ali_order_id = df_ali_order.订单ID.astype(str).tolist()

df_all = df_all[(df_all.归属渠道=='芝麻租物')&(df_all.下单日期=='2025-12-17')]
# df_yx_ckq = select_bf_data()
# change_type == 'PASS_TIME'，初审通过，狭义上的支付成功，约等于发起支付的时间
df_all.loc[:, '是否48h'] = np.where((df_all.go_express_date.dt.date <= (df_all.create_time.dt.date + pd.Timedelta(days=1))) & (df_all.change_type == 'PASS_TIME'), 1, 0)
# df_all.loc[:, '七天内'] = np.where((df_all.go_express_date - df_all.支付时间 <= pd.Timedelta(days=7))&(df_all.change_type=='PASS_TIME'), 1, 0)
# df_all_merge = df_all.merge(df_yx_ckq[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
df_all_merge = df_all.copy()
df_all_merge.loc[:, '优质订单'] = np.where((df_all_merge.can_ship_flag==True)&(df_all_merge.is_bf_3h==True)&(df_all_merge.is_bf_6h==True)&(df_all_merge.is_bf_12h==True)&(df_all_merge.is_bf_24h==True)&(df_all_merge.是否出库前风控强拒==0), 1, 0)
# 优质订单记录
# df_all_yz = df_all_merge.copy()#[df_all_merge.优质订单==1]
df_all_yz = df_all_merge[df_all_merge.优质订单==1]
df_alipay_id = df_all_yz.alipay_order_id.astype(str).tolist()
# 通过订单ID(alipay_order_id)联系，审核通过时间即为pass_time传递的create_time
# 找出df_ali_order_id有而df里没有的订单记录
df_ali_not_in_df = df_ali_order[~df_ali_order.订单ID.isin(df_alipay_id)]
print(df_ali_not_in_df.shape)
# 找出df_ali_order_id没有而df里有的订单记录
df_not_in_ali = df_all_yz[~df_all_yz.alipay_order_id.isin(df_ali_order_id)]
print(df_not_in_ali.shape)


(9, 30)
(1, 131)


In [90]:
df_ali_not_in_df[['订单ID']]
# 其它渠道订单

,订单ID
2,2025121701502300000008620080610965
7,2025121701502300000009580054410595
13,2025121701502300000003090012193470
23,2025121701502300000003090012423423
75,2025121701502300000007950043973209
97,2025121701502300000009160061099155
105,2025121701502300000007240020215576
112,2025121701502300000008630005719190
179,2025121701502300000004550077251784


In [ ]:
df_not_in_ali

In [92]:
df_all_yz.loc[:, '是否优质订单支付2天内发货笔数'] = np.where((df_all_yz.是否48h==1)&(df_all_yz.是否出库==1), 1, 0)

# df_all_yz.loc[:, '是否优质订单支付7天内发货'] = np.where((df_all_yz.七天内==1)&(df_all_yz.是否出库==1), 1, 0)
df_all_yz_merge = df_all_yz[['order_number', 'alipay_order_id', '优质订单','是否优质订单支付2天内发货笔数','下单日期', 'create_time', 'merchant_name','status2','是否出库']].merge(df_ali_order, left_on='alipay_order_id',right_on='订单ID', how='outer')
df_all_yz_merge.to_excel('F:\需求\总需求\下单时间迪瓜优质订单1217核验.xlsx', index=False)

In [39]:
df_all_yz.groupby('下单日期').agg({'是否进件':'sum', '是否优质订单支付2天内发货笔数':'sum', '是否优质订单支付7天内发货':'sum'})

,是否进件,是否优质订单支付2天内发货笔数,是否优质订单支付7天内发货
下单日期,,,
2025-12-01,220,34,34
2025-12-02,254,38,38
2025-12-03,248,43,45
2025-12-04,250,42,42
2025-12-05,237,33,33
2025-12-06,183,28,28
2025-12-07,170,24,24
2025-12-08,251,32,32
2025-12-09,311,35,35


In [ ]:
# df_ali_not_in_df_t = df_ali_not_in_df.merge(df_order[['alipay_order_id', 'order_number']], how='left', left_on='订单ID', right_on='alipay_order_id')
# df_ali_not_in_df_t.drop(['alipay_order_id'], axis=1, inplace=True)
# df_ali_not_in_df_t[['alipay_order_id', 'order_number']]

In [ ]:
df_all_yz[df_all_yz.是否优质订单支付7天内发货==1][['order_number', 'alipay_order_id', '是否优质订单支付2天内发货笔数', '是否优质订单支付7天内发货', '下单日期', 'merchant_name','status2','是否出库','七天内']]

## 核验全量支付订单

In [125]:
df_ali_order = pd.read_excel('F:\需求\总需求\租赁阵地-订单支付明细-迪瓜1201-1210.xlsx')
df_ali_order_id = df_ali_order.订单ID.astype(str).tolist()


In [129]:

df_all = df_all[(df_all.归属渠道=='芝麻租物')&(df_all.是否进件==1)]
df_yx_ckq = select_bf_data()
df_all.loc[:, '是否48h'] = np.where((df_all.go_express_date.dt.date <= (df_all.create_time.dt.date + pd.Timedelta(days=1))) & (df_all.change_type == 'PASS_TIME'), 1, 0)
# df_all.loc[:, '七天内'] = np.where((df_all.go_express_date - df_all.支付时间 <= pd.Timedelta(days=7))&(df_all.change_type=='PASS_TIME'), 1, 0)
df_all_merge = df_all.merge(df_yx_ckq[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
df_all_merge.loc[:, '优质订单'] = np.where((df_all_merge.is_bf==True)&(df_all_merge.can_ship_flag==True), 1, 0)
# 优质订单记录
df_all_yz = df_all_merge#[df_all_merge.优质订单==0]
df_alipay_id = df_all_yz.alipay_order_id.astype(str).tolist()
# 通过订单ID(alipay_order_id)联系，审核通过时间即为pass_time传递的create_time
# 找出df_ali_order_id有而df里没有的订单记录
df_ali_not_in_df = df_ali_order[~df_ali_order.订单ID.isin(df_alipay_id)]
print(df_ali_not_in_df.shape)
# 找出df_ali_order_id没有而df里有的订单记录
df_not_in_ali = df_all_yz[~df_all_yz.alipay_order_id.isin(df_ali_order_id)]
print(df_not_in_ali.shape)

(50, 30)
(5547, 125)


In [ ]:
df_all_yz[df_all_yz.alipay_order_id=='2025120301502300000002520044378299']

In [131]:
df_all_yz.loc[:, '是否支付2天内发货笔数'] = np.where((df_all_yz.是否48h==1)&(df_all_yz.是否出库==1), 1, 0)
# df_all_yz.loc[:, '是否支付7天内发货'] = np.where((df_all_yz.七天内==1)&(df_all_yz.是否出库==1), 1, 0)

In [133]:
df_all_merge_m = df_all_yz[['order_number', 'alipay_order_id', '优质订单','是否支付2天内发货笔数','下单日期', 'go_express_date','merchant_name','status2','是否出库']].merge(df_ali_order, left_on='alipay_order_id',right_on='订单ID', how='outer')
df_all_merge_m.to_excel('F:\需求\总需求\迪瓜全量订单1201-1210核验2.xlsx', index=False)

In [64]:
df_all[df_all.order_number=='A2025120709210857'][['归属渠道','order_number','alipay_order_id','status2','can_ship_flag','是否拒量','merchant_name']]

,归属渠道,order_number,alipay_order_id,status2,can_ship_flag,是否拒量,merchant_name
1233,其他渠道场景渠道。,A2025120709210857,2025120701502300000005920023998740,已退款,None,0,维客自营账户


In [82]:
df_not_in_ali[['alipay_order_id']]

,alipay_order_id
51,2025120101502300000005930091496715
73,2025120101502300000003910039614148
132,2025120101502300000002800027417154
134,2025120101502300000002800027406076
155,2025120101502300000004220062464743
...,...
13193,2025121001502300000003100030772317
13196,2025121001502300000003650051943681
13260,2025121001502300000004260023384075
13270,2025121001502300000000260099161212


In [68]:
df_not_in_ali[['order_number', 'merchant_name', 'alipay_order_id','status2','归属渠道','can_ship_flag','is_bf','优质订单']].shape

(3349, 8)

In [37]:
df_all_yz.loc[:, '是否优质订单支付2天内发货笔数'] = np.where((df_all_yz.是否48h==1)&(df_all_yz.是否出库==1), 1, 0)
df_all_yz.groupby('下单日期').agg({'是否进件':'sum', '优质订单支付2天内发货笔数':'sum'})

,是否进件,优质订单支付2天内发货笔数
下单日期,,
2025-12-07,170,24


(170, 38)

In [29]:
# # 支付宝渠道11.19-12.8自营进件，按照身份证+进件日期去重，打客群标签（活动商品下单出库必发、非活动商品下单出库必发、其他）
# df_yx_ckq = select_bf_data()
# df_all_merge = df_all.merge(df_yx_ckq[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
# df_all_merge = df_all_merge[(df_all_merge.是否进件==1)&(df_all_merge.can_ship_flag==1)]
# df_all_merge.loc[:, '客群标签'] = np.where((df_all_merge.is_bf==True)&(df_all_merge.严选活动商品=='是'), '活动商品下单出库必发', 
#                                         np.where((df_all_merge.is_bf==True)&(df_all_merge.严选活动商品=='否'), '非活动商品下单出库必发', '其他'))
# df_all_merge.shape

(3210, 121)

In [ ]:
# 日期：12.1-12.7
# 优质订单：每日/周进件出库率数据汇总、进件明细、出库明细
# 整体订单：每日/周进件出库率数据汇总、进件明细、出库明细
# 字段：支付宝订单号、订单编号、进件时间、发货时间、下单是必发标签、最终必发标签
df_all = df_all[df_all.归属渠道=='芝麻租物']
df_yx_ckq = select_bf_data()
# change_type=='PASS_TIME'，初审通过，狭义上的支付成功
df_all.loc[:, '是否48h'] = np.where((df_all.go_express_date - df_all.支付时间 <= pd.Timedelta(hours=48))&(df_all.change_type=='PASS_TIME'), 1, 0)
df_all.loc[:, '支付数'] = np.where(df_all.change_type=='PASS_TIME', 1, 0)
df_all_merge = df_all.merge(df_yx_ckq[['out_biz_no', 'is_bf']], left_on='order_number',right_on='out_biz_no', how='left')
df_all_merge.loc[:, '优质订单'] = np.where((df_all_merge.is_bf==True)&(df_all_merge.can_ship_flag==True), 1, 0)
df_all_merge_bf = df_all_merge[df_all_merge.优质订单==1]
df_bf_g = df_all_merge_bf.groupby('下单日期').agg({'是否进件':'sum', '是否出库':'sum','是否48h':'sum','支付数':'sum'})
df_bf_g['发货率'] = (df_bf_g['是否48h']/df_bf_g['支付数']).apply(lambda x: format(x,'.2%'))
df_bf_g['出库率'] = (df_bf_g['是否出库']/df_bf_g['是否进件']).apply(lambda x: format(x,'.2%'))
df_bf_g

In [181]:
df_all_merge_bf[df_all_merge_bf.change_type=='PASS_TIME'][['order_number','支付数','是否48h', 'go_express_date', '支付时间','下单日期','change_type']]

,order_number,支付数,是否48h,go_express_date,支付时间,下单日期,change_type


In [ ]:
df_all_merge_bf = df_all_merge_bf[['alipay_order_id', 'order_number', 'create_time', 'go_express_date', 'can_ship_flag', 'is_bf', 'status2', '是否48h','支付时间']]
df_all_merge_bf

In [ ]:
df_all_meger_ibf = df_all_merge[df_all_merge.优质订单==0]
df_ibf_g = df_all_meger_ibf.groupby('下单日期').agg({'是否进件':'sum', '是否出库':'sum','是否48h':'sum','支付数':'sum'})
df_ibf_g['发货率'] = (df_ibf_g['是否48h']/df_ibf_g['支付数']).apply(lambda x: format(x,'.2%'))
df_ibf_g['出库率'] = (df_ibf_g['是否出库']/df_ibf_g['是否进件']).apply(lambda x: format(x,'.2%'))
df_ibf_g

In [37]:
df_all_meger_ibf= df_all_meger_ibf[df_all_meger_ibf.是否进件==1][['alipay_order_id', 'order_number', 'create_time', 'go_express_date', 'can_ship_flag', 'is_bf', 'status2', '是否48h','支付时间']]
df_all_meger_ibf.shape

(6137, 9)

In [38]:
# with pd.ExcelWriter(f'F:\需求\瑞彤需求\严选优质订单明细数据2_1201-1207.xlsx') as writer:
#     df_bf_g.to_excel(writer, sheet_name='优质订单进件出库率')
#     df_all_merge_bf.to_excel(writer, sheet_name='优质订单明细', index=False)
#     df_ibf_g.to_excel(writer, sheet_name='整体订单进件出库率')
#     df_all_meger_ibf.to_excel(writer, sheet_name='整体订单明细', index=False)